# YOLOv8 Training for Vehicle Detection
This notebook demonstrates how to fine-tune YOLOv8 for traffic vehicle detection

## Dataset
We'll use a vehicle detection dataset from Kaggle:
- **Dataset Link**: https://www.kaggle.com/datasets/sshikamaru/car-object-detection
- Alternative: https://www.kaggle.com/datasets/dataclusterlabs/vehicle-detection-image-dataset

## Requirements
- ultralytics
- PyTorch
- OpenCV

In [ ]:
# Install required packages
!pip install ultralytics torch torchvision opencv-python

In [ ]:
from ultralytics import YOLO
import os
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## Step 1: Download and Prepare Dataset

### Dataset Structure
```
vehicle_dataset/
├── images/
│   ├── train/
│   └── val/
└── labels/
    ├── train/
    └── val/
```

### Download from Kaggle
1. Install Kaggle API: `pip install kaggle`
2. Setup Kaggle credentials: Place `kaggle.json` in `~/.kaggle/`
3. Download dataset:
```bash
kaggle datasets download -d sshikamaru/car-object-detection
unzip car-object-detection.zip -d vehicle_dataset
```

In [ ]:
# Create dataset configuration file
dataset_yaml = """
path: /path/to/vehicle_dataset  # Update this path
train: images/train
val: images/val

# Classes
nc: 4  # number of classes
names: ['car', 'motorcycle', 'bus', 'truck']  # class names
"""

with open('vehicle_data.yaml', 'w') as f:
    f.write(dataset_yaml)

print("Dataset configuration created: vehicle_data.yaml")

## Step 2: Load Pretrained YOLOv8 Model

In [ ]:
# Load a pretrained YOLOv8 model
# Options: yolov8n.pt (nano), yolov8s.pt (small), yolov8m.pt (medium), yolov8l.pt (large)
model = YOLO('yolov8n.pt')  # nano version for faster training

print("Model loaded successfully!")
print(f"Model architecture: {model.model}")

## Step 3: Train the Model

In [ ]:
# Training configuration
results = model.train(
    data='vehicle_data.yaml',  # path to dataset YAML
    epochs=50,                  # number of training epochs
    imgsz=640,                  # image size
    batch=16,                   # batch size (adjust based on GPU memory)
    name='vehicle_detection',   # project name
    patience=10,                # early stopping patience
    save=True,                  # save checkpoints
    device='mps',               # use 'mps' for Apple Silicon, 'cuda' for NVIDIA GPU, 'cpu' for CPU
    workers=4,                  # number of dataloader workers
    pretrained=True,            # use pretrained weights
    optimizer='Adam',           # optimizer (Adam, SGD, etc.)
    lr0=0.001,                  # initial learning rate
    lrf=0.01,                   # final learning rate
    momentum=0.937,             # momentum
    weight_decay=0.0005,        # weight decay
    warmup_epochs=3,            # warmup epochs
    warmup_momentum=0.8,        # warmup momentum
    box=7.5,                    # box loss gain
    cls=0.5,                    # classification loss gain
    plots=True                  # save training plots
)

print("Training completed!")

## Step 4: Validate the Model

In [ ]:
# Validate the trained model
metrics = model.val()

print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")

## Step 5: Test Inference

In [ ]:
# Test on a sample image
results = model.predict(
    source='path/to/test/image.jpg',
    save=True,
    conf=0.25,  # confidence threshold
    iou=0.45    # NMS IOU threshold
)

# Display results
for result in results:
    print(f"Detected {len(result.boxes)} objects")
    result.show()

## Step 6: Export Model

In [ ]:
# Export trained model to different formats

# Export to ONNX (for deployment)
model.export(format='onnx')

# Export to TorchScript (for PyTorch deployment)
model.export(format='torchscript')

# Export to CoreML (for iOS/macOS)
model.export(format='coreml')

print("Model exported successfully!")

## Step 7: Save Best Weights

In [ ]:
# The best weights are automatically saved during training
# Location: runs/detect/vehicle_detection/weights/best.pt

import shutil

# Copy best weights to backend folder
source = 'runs/detect/vehicle_detection/weights/best.pt'
destination = '../backend/yolov8_custom.pt'

shutil.copy(source, destination)
print(f"Best weights copied to {destination}")

## Alternative: Using Pretrained COCO Model

If you don't want to train a custom model, you can use the pretrained YOLOv8 model trained on COCO dataset which already includes vehicle classes:
- car (class 2)
- motorcycle (class 3)
- bus (class 5)
- truck (class 7)

Simply use `yolov8n.pt`, `yolov8s.pt`, or `yolov8m.pt` directly in your application!

In [ ]:
# Download and save pretrained model
pretrained_model = YOLO('yolov8n.pt')
print("Pretrained model ready to use!")
print("Classes:", pretrained_model.names)